In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer
import tensorflow as tf
from pathlib import Path
from time import strftime

In [2]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs
print('Tensorboard launched at localhost:6006')

Launching TensorBoard...

Tensorboard launched at localhost:6006


In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
def get_run_dir(root='my_logs'):
    return Path(root) / strftime('run_%Y_%m_%d_%H_%M_%S')

In [5]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

In [6]:
dataset = pd.read_csv('dataset.csv')

In [7]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]

In [8]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling input values to Gaussian distributions

In [9]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

# Training

In [10]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(1)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization (Normalizatio  (None, 8)                17        
 n)                                                              
                                                                 
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                             

2022-11-18 22:07:04.306449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 22:07:04.919614: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 34 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1


In [11]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=['mae'])

In [12]:
norm_layer.adapt(data_train_qt)

In [13]:
n_epochs = 100

run_dir = get_run_dir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

history = model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation), callbacks=[tensorboard_cb])

Epoch 1/100
570/570 [==============================] - 3s 4ms/step - loss: 5.9282 - mae: 2.0104 - val_loss: 7.5130 - val_mae: 2.3646
Epoch 2/100
570/570 [==============================] - 2s 4ms/step - loss: 4.4219 - mae: 1.7918 - val_loss: 7.2038 - val_mae: 2.3148
Epoch 3/100
570/570 [==============================] - 2s 4ms/step - loss: 4.3993 - mae: 1.7868 - val_loss: 6.9233 - val_mae: 2.2666
Epoch 4/100
570/570 [==============================] - 2s 4ms/step - loss: 4.3943 - mae: 1.7862 - val_loss: 6.6696 - val_mae: 2.2194
Epoch 5/100
570/570 [==============================] - 2s 4ms/step - loss: 4.3913 - mae: 1.7853 - val_loss: 6.8644 - val_mae: 2.2560
Epoch 6/100
570/570 [==============================] - 2s 4ms/step - loss: 4.3897 - mae: 1.7847 - val_loss: 7.0523 - val_mae: 2.2892
Epoch 7/100
570/570 [==============================] - 2s 4ms/step - loss: 4.3881 - mae: 1.7845 - val_loss: 6.7013 - val_mae: 2.2255
Epoch 8/100
570/570 [==============================] - 2s 4ms/step - 

In [14]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 0s 2ms/step

Values for validation set:
MAE: 1.6895632124705573
MSE: 4.379270255323104
RMSE: 2.0926706036362015
R2:  -0.10084465634301365


# Overfitting?

In [15]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(1)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization_1 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 26
Trainable params: 9
Non-trainable params: 17
_________________________________________________________________


In [16]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [17]:
norm_layer.adapt(data_train_qt)

In [18]:
n_epochs = 50

# run_dir = get_run_dir()
# tensorboard_cb = tf.keras.callbacks.TensorBoard(run_dir, profile_batch=(0, n_epochs))

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
570/570 [==============================] - 2s 4ms/step - loss: 30.1346 - mae: 5.0169 - val_loss: 30.5456 - val_mae: 4.9448
Epoch 2/50
570/570 [==============================] - 2s 4ms/step - loss: 24.5595 - mae: 4.4737 - val_loss: 22.7860 - val_mae: 4.1177
Epoch 3/50
570/570 [==============================] - 2s 4ms/step - loss: 20.1063 - mae: 3.9677 - val_loss: 19.5052 - val_mae: 3.7832
Epoch 4/50
570/570 [==============================] - 2s 3ms/step - loss: 16.4159 - mae: 3.5166 - val_loss: 15.8962 - val_mae: 3.4003
Epoch 5/50
570/570 [==============================] - 2s 3ms/step - loss: 13.3604 - mae: 3.1323 - val_loss: 13.3055 - val_mae: 3.1107
Epoch 6/50
570/570 [==============================] - 2s 3ms/step - loss: 10.8655 - mae: 2.8094 - val_loss: 11.5084 - val_mae: 2.9008
Epoch 7/50
570/570 [==============================] - 2s 3ms/step - loss: 8.8733 - mae: 2.5426 - val_loss: 9.6568 - val_mae: 2.6697
Epoch 8/50
570/570 [==============================] - 2s 3ms/ste

In [19]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

215/215 [==============================] - 0s 2ms/step

Values for validation set:
MAE: 1.683327297864196
MSE: 4.350483806243208
RMSE: 2.0857813419060034
R2:  -0.09360842592170626


# Check if data from all subjects works better

In [20]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [21]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

In [22]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
output = tf.keras.layers.Dense(1)(normalized)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization_2 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 26
Trainable params: 9
Non-trainable params: 17
_________________________________________________________________


In [23]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [24]:
norm_layer.adapt(data_train_qt)

In [25]:
n_epochs = 50

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/50
2394/2394 [==============================] - 7s 3ms/step - loss: 22.3546 - mae: 4.1928 - val_loss: 15.0815 - val_mae: 3.3113
Epoch 2/50
2394/2394 [==============================] - 7s 3ms/step - loss: 9.5316 - mae: 2.6218 - val_loss: 8.2039 - val_mae: 2.4706
Epoch 3/50
2394/2394 [==============================] - 7s 3ms/step - loss: 5.0278 - mae: 1.9482 - val_loss: 6.3221 - val_mae: 2.1255
Epoch 4/50
2394/2394 [==============================] - 7s 3ms/step - loss: 4.3890 - mae: 1.7937 - val_loss: 6.1984 - val_mae: 2.0880
Epoch 5/50
2394/2394 [==============================] - 7s 3ms/step - loss: 4.3801 - mae: 1.7831 - val_loss: 6.2068 - val_mae: 2.0916
Epoch 6/50
2394/2394 [==============================] - 7s 3ms/step - loss: 4.3802 - mae: 1.7837 - val_loss: 6.2030 - val_mae: 2.0901
Epoch 7/50
2394/2394 [==============================] - 7s 3ms/step - loss: 4.3799 - mae: 1.7831 - val_loss: 6.2088 - val_mae: 2.0923
Epoch 8/50
2394/2394 [==============================] - 7s 3

In [26]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 2s 2ms/step

Values for validation set:
MAE: 1.704908590704877
MSE: 4.467483340981309
RMSE: 2.1136421979562456
R2:  -0.09759174512293645


Trying big model again

In [27]:
tf.random.set_seed(123)

norm_layer = tf.keras.layers.Normalization()

input_ = tf.keras.layers.Input(shape=data_train_qt.shape[1:])
normalized = norm_layer(input_)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(normalized)
hidden2 = tf.keras.layers.Dense(128, activation='relu')(hidden1)
hidden3 = tf.keras.layers.Dense(64, activation='relu')(hidden2)
output = tf.keras.layers.Dense(1)(hidden3)

model = tf.keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8)]               0         
                                                                 
 normalization_3 (Normalizat  (None, 8)                17        
 ion)                                                            
                                                                 
 dense_6 (Dense)             (None, 64)                576       
                                                                 
 dense_7 (Dense)             (None, 128)               8320      
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                           

In [28]:
model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])
norm_layer.adapt(data_train_qt)

In [29]:
n_epochs = 25

model.fit(data_train_qt, labels_train, epochs=n_epochs, validation_data=(data_validation, labels_validation))

Epoch 1/25
2394/2394 [==============================] - 7s 3ms/step - loss: 4.6072 - mae: 1.8162 - val_loss: 7.0074 - val_mae: 2.2782
Epoch 2/25
2394/2394 [==============================] - 7s 3ms/step - loss: 4.4085 - mae: 1.7866 - val_loss: 6.7988 - val_mae: 2.2391
Epoch 3/25
2394/2394 [==============================] - 9s 4ms/step - loss: 4.4003 - mae: 1.7850 - val_loss: 6.8114 - val_mae: 2.2417
Epoch 4/25
2394/2394 [==============================] - 8s 3ms/step - loss: 4.3969 - mae: 1.7848 - val_loss: 6.4611 - val_mae: 2.1655
Epoch 5/25
2394/2394 [==============================] - 6s 3ms/step - loss: 4.3914 - mae: 1.7836 - val_loss: 7.5778 - val_mae: 2.3746
Epoch 6/25
2394/2394 [==============================] - 7s 3ms/step - loss: 4.3915 - mae: 1.7838 - val_loss: 6.8399 - val_mae: 2.2471
Epoch 7/25
2394/2394 [==============================] - 9s 4ms/step - loss: 4.3874 - mae: 1.7835 - val_loss: 6.8404 - val_mae: 2.2472
Epoch 8/25
2394/2394 [==============================] - 7s 3ms

In [30]:
predictions = model.predict(data_validation_qt)

mae, mse, r2 = get_score(labels_validation.iloc[:, 1], predictions)
print(f'''\nValues for validation set:\nMAE: {mae}\nMSE: {mse}\nRMSE: {mse**.5}\nR2:  {r2}''')

895/895 [==============================] - 1s 1ms/step

Values for validation set:
MAE: 1.6441731698019406
MSE: 4.2095829292362845
RMSE: 2.051726816424712
R2:  -0.034229592118697916
